In [2]:
!pip3 install brayns==1.0.0

  Using cached https://files.pythonhosted.org/packages/47/5c/6594e0ac63e4ff6e70a06326d0575d6c9992a96aff1549c0032e0b56070a/brayns-1.0.0.tar.gz
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/ae/2a/0a0ab2833e5270664fb5fae590717f867ac6319b124160c09f1d3291de28/Pillow-5.4.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a3/9f/43df3d9c4d9df60dfca0a161461835514209e8405bba4d6aba36b1da5652/python_jsonschema_objects-0.3.12-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f0/ec/561ec49e2ef7e089c5d3088aa48ef2554e3c5d7d5074452d727c06d91596/rockets-1.0.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/21/18/a0de8cda637ba3efee1b3617ded00601507ce15bd70a393

# Allocate and start Brayns on BB5

# Start scripting!

In [3]:
from brayns import Client, __version__ as brayns_version

In [4]:
print('brayns:   ' + brayns_version)

brayns:   1.0.0


In [5]:
brayns = Client('r1i7n21.bbp.epfl.ch:5000')

In [6]:
brayns.get_model_properties(id=0)
brayns.add_model()

In [78]:
filename = "/gpfs/bbp.cscs.ch/project/proj3/.tolokoban/"
#brayns.image(size=(384,216), format='jpeg', quality=100, samples_per_pixel=64).save(filename)
ce.export_frames_to_disk(path=filename,animation_frames=[757],samples_per_pixel=64)

TypeError: export_frames_to_disk() missing 1 required positional argument: 'camera_definitions'

## Add mouse brain model

In [18]:
brayns.add_model(
    path='/gpfs/bbp.cscs.ch/project/proj3/resources/meshes/mouse_brain_and_column/full/mouse_brain_and_column.ply')

{'bounding_box': False,
 'bounds': {'max': [-1.7976931348623157e+308,
   -1.7976931348623157e+308,
   -1.7976931348623157e+308],
  'min': [1.7976931348623157e+308,
   1.7976931348623157e+308,
   1.7976931348623157e+308]},
 'id': 7,
 'metadata': {'faces': '881968', 'meshes': '1', 'vertices': '1763936'},
 'name': 'mouse_brain_and_column',
 'path': '/gpfs/bbp.cscs.ch/project/proj3/resources/meshes/mouse_brain_and_column/full/mouse_brain_and_column.ply',
 'transformation': {'rotation': [0.0, 0.0, 0.0, 1.0],
  'rotation_center': [0.0, 0.0, 0.0],
  'scale': [1.0, 1.0, 1.0],
  'translation': [0.0, 0.0, 0.0]},
 'visible': True}

## List models

In [58]:
brayns.scene.models[0]["id"]

5

## Materials

In [62]:
id=brayns.scene.models[0]["id"]
circuit_explorer.set_material_extra_attributes(model_id=id)
circuit_explorer.set_material(
    glossiness=1, opacity=1,
    model_id=id, material_id=0, 
    diffuse_color=(1,0,0),
    shading_mode=circuit_explorer.SHADING_MODE_DIFFUSE)

'OK'

## Shading options

In [ ]:
params = brayns.AdvancedSimulationRendererParams()
params.shadows=1
params.soft_shadows=1
params.pixel_alpha=2
brayns.set_renderer_params(params)

In [ ]:
brayns.set_environment_map('/gpfs/bbp.cscs.ch/project/proj3/resources/envmap/0002.jpg')

In [ ]:
img = brayns.image(
    format='png', quality=100, samples_per_pixel=16, size=(800,600))

In [72]:
input = {
    "host":"r1i7n13:5000",
    "modelId":brayns.scene.models[0]["id"],
    "materialId":0,
    "opacity":1,
    "glossiness":1,
    "diffuseColor":[1,0,0],
    "shadingMode":"cartoon"
}

print(input)

class Error(Exception):
    def __init__(self, code, text):
        self.code = code
        self.text = text


class MissingInput(Error):
    def __init__(self, att_name):
        self.code = 1
        self.text = f"Missing mandatory input attribute: \"{att_name}\"!"

def exec(input):
    """
    Set a material for a model.

    IN: {
      host: string,
      modelId: number,
      materialId: number,
      diffuseColor: [number,number,number],
      shadingMode: "none"|  "diffuse" | "cartoon" | "electron",
      glossiness?: number,
      opacity?: number
    }
    """
    for att_name in ("host", "modelId", "materialId", "diffuseColor", "shadingMode"):
        if att_name not in input:
            raise MissingInput(att_name)

    brayns = Client(input["host"])
    circuit_explorer = CircuitExplorer(brayns)

    circuit_explorer.set_material_extra_attributes(model_id=input["modelId"])

    shading_mode = input["shadingMode"]
    if shading_mode == "diffuse":
        shading_mode = CircuitExplorer.SHADING_MODE_DIFFUSE
    elif shading_mode == "cartoon":
        shading_mode = CircuitExplorer.SHADING_MODE_CARTOON
    elif shading_mode == "electron":
        shading_mode = CircuitExplorer.SHADING_MODE_ELECTRON
    else:
        shading_mode = CircuitExplorer.SHADING_MODE_NONE

    if "glossiness" not in input:
        input["glossiness"] = 0
    if "opacity" not in input:
        input["opacity"] = 1

    circuit_explorer.set_material(
        glossiness=input["glossiness"],
        opacity=input["opacity"],
        model_id=input["modelId"],
        material_id=input["materialId"],
        diffuse_color=input["diffuseColor"],
        shading_mode=shading_mode)

    return True

exec(input)

{'host': 'r1i7n13:5000', 'modelId': 13, 'materialId': 0, 'opacity': 1, 'glossiness': 1, 'diffuseColor': [1, 0, 0], 'shadingMode': 'cartoon'}


True

## Add grid

In [ ]:
circuit_explorer.add_grid(colored=True, show_axis=False, opacity=0.5,
    min_value=-20000, max_value=20000, interval=1000)

## Camera path

In [ ]:
control_points = list()

In [ ]:
control_points.append(circuit_explorer.get_camera())

In [ ]:
control_points

In [ ]:
camera_path = CameraPathHandler(control_points, 100, 100)

In [ ]:
camera_path.get_key_frame(10)

In [ ]:
for i in range(camera_path.get_nb_frames()):
#     import time
#     time.sleep(0.1)
    kf = camera_path.get_key_frame(i)
    circuit_explorer.set_camera(origin=kf[0], direction=kf[1], up=kf[2])

## Export movie

In [ ]:
animation_frames=list()
camera_definitions = list()
for i in range(camera_path.get_nb_frames()):
    animation_frames.append(0)
    camera_definitions.append(camera_path.get_key_frame(i))

In [ ]:
brayns.set_application_parameters(viewport=[800,600])

In [ ]:
circuit_explorer.export_frames_to_disk(
    animation_frames=animation_frames, 
    camera_definitions=camera_definitions, 
    path='/gpfs/bbp.cscs.ch/project/proj3/projects/mopro/tmp/normals/', 
    samples_per_pixel=1)

In [ ]:
circuit_explorer.cancel_frames_export()